In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import spacy
# nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim.downloader
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer

import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, optimizers, layers
from keras.preprocessing import text, sequence
from keras.utils import plot_model

!pip install -q -U keras-tuner
import kerastuner as kt

import IPython


from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import re

from nltk import pos_tag
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer 

from gensim.models import word2vec

from IPython.display import display 

import seaborn as sns
import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

sns.set(context = 'notebook', style = 'whitegrid')
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows',50)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\caell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\caell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\caell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
df = pd.read_csv('../data/processed_text.csv')
X=df.Text
y=df.Grade

Below text thanks to:https://www.kaggle.com/jjjohnson/experiments-sentiment-analysis-raw-data-gru-lstm/edit

https://towardsdatascience.com/hands-on-nlp-deep-learning-model-preparation-in-tensorflow-2-x-2e8c9f3c7633

GloVe embeddigns thanks to Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]




In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)
X_t, X_val, y_val, y_val = train_test_split(X_train, y_train, random_state = 42, test_size = 0.2)

total_vocabulary = set()
for text in X_t:
    words = text.split(' ')
    total_vocabulary = total_vocabulary.union(words)
num_words = len(total_vocabulary)
print(f'total vocabulary length: {num_words}')


tokenizer = Tokenizer()

tokenizer.fit_on_texts(X_t)

word_index = tokenizer.word_index

glove = {}
with open('glove.6B.50d.txt',
          'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype = np.float32)
            glove[word] = vector


num_tokens = len(total_vocabulary) + 2
embedding_dim = 50
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = glove.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits +=1
    else:
        misses += 1
print(f'converted {hits} words ({misses} misses)')
    

X_t_glove = vectorizer(np.array([[s] for s in X_t])).numpy()
X_val_glove = vectorizer(np.array([[s] for s in X_val])).numpy()
X_test_glove = vectorizer(np.array([[s] for s in X_test])).numpy()
X_train_glove = vectorizer(np.array([[s] for s in X_train])).numpy()

total vocabulary length: 13727
converted 7223 words (6504 misses)


In [ ]:

feature = 'Tokens'
X = df[feature]
y= df.Grade
samples = df[['Grade',feature]]
train_set_size = .8
idx=np.arange(len(X))
np.random.shuffle(idx)
split_point = int(len(idx)*train_set_size)
train_samples = samples.iloc[idx[:split_point]]
test_samples = samples.iloc[idx[split_point:]]


vectorizer = TextVectorization(output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples.to_numpy()).batch(64)
vectorizer.adapt(text_ds)
# X_train=vectorizer(np.array([[s] for s in train_samples])).numpy()
# y_train = y[idx[:split_point]]

# X_test=vectorizer(np.array([[s] for s in test_samples])).numpy()

# y_test = y[idx[split_point:]]
# X_test

In [ ]:
word2vec = Word2Vec(df.Tokens, min_count=1)
vocabulary = word2vec.wv.vocab

In [ ]:
word2vec.wv.most_similar('school')

In [ ]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

Thanks to https://keras.io/examples/nlp/pretrained_word_embeddings/ for the below

In [ ]:
words = vocabulary.keys()
word_index = dict(zip(words, range(len(words))))

In [ ]:
len(word_index.keys())

In [ ]:
word_index['chobot']

In [ ]:
num_tokens = len(words)+2
embedding_dim = len(glove_vectors['anything'])
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    
    try: 
        embedding_vector = glove_vectors[word]
        embedding_matrix[i] = embedding_vector
        hits += 1
    except:
        misses +=1
print('Converted %d words (%d misses)' % (hits, misses))

In [ ]:
embedding_matrix.shape

In [ ]:
from tensorflow.keras.layers import Embedding
embedding_layer=Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False)

In [ ]:
from tensorflow.keras import layers
int_sequences_input = keras.Input(shape=(None,),dtype="int64")
embedded_sequences=embedding_layer(int_sequences_input)
x=layers.LSTM(10,activation='relu',recurrent_activation='relu')(embedded_sequences)
preds=layers.Dense(1, activation='relu')(x)
model = keras.Model(int_sequences_input, preds)

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae'])
model.summary()



In [ ]:
X_t = np.asarray(X_train).astype('float32')

In [ ]:
X_t = word2vec.transform(np.array([[s] for s in df.Tokens])).numpy()

In [ ]:
train_samples

In [ ]:
model.fit(X_train, y_train,
         batch_size=1,
         epochs=10,
         )

In [ ]:
X_train